In [4]:

class ColumnNames:
    def __init__(self,columnName, dataType, isnull):
       
        self.ColumnName = columnName
        self.DataType = dataType
        self.IsNull = isnull
    
    def display_info(self):
        return f"{self.columnName} {self.dataType} {self.IsNull}"
    
class DDL:
    def __init__(self,TableName,columns:list[ColumnNames]):
        self.TableName = TableName
        self.columns=columns
    def display_info(self):
        return f"{self.TableName}"




    

In [6]:
f = open(".\\ddl.txt", "r")
#print(f.readlines())
readTablesDLL=f.read()
getDDLAsList=readTablesDLL.split("CREATE")[1:len(readTablesDLL.split("CREATE"))+1]
#len(getDDLAsList)
#print(len(getDDLAsList))
#print(getDDLAsList)
DDLList=[]
for tables in getDDLAsList:
  
    #get Table Name
    tableNamestring=tables[tables.find("TABLE"):tables.find("(")]
    tableNameArray=tableNamestring.split(".")
    
    tableName=tableNameArray[1][1:tableNameArray[1].find(']')]
    #print(tableName)
    #print(tableNameArray[1].find(']'))
    #get Column,Datatype and isnull details
    startindex=tables.find("(")+1
    endindex=tables.rfind(")")
    GetSingleTableColumnDetails=tables[startindex:endindex]
    getTableColumnDatatypestring=GetSingleTableColumnDetails.split('\n')
    getColumns=getTableColumnDatatypestring[1:-1]
    #print(getColumns)
    colList=[]
    for s in getColumns:
        #print(s)
        tempcollist=s.split('\t')[1:]
        collist=tempcollist
        #print(type(tempcollist))
        #print(tempcollist)
        #for col in collist:
            
     
        colnames=collist[0].split(' ')
        #print(len(colnames))
        if(len(colnames)>=5):
               
            colName=colnames[0][1:colnames[0].find(']')]
            #print(colName)
            datatpe=colnames[1][1:colnames[1].find(']')]
            #print(datatpe)
            isnul=colnames[3]
            if(isnul.find(",")>-1):
                isnul=isnul[0:-1]
            
            colitem=ColumnNames(columnName=colName,dataType=datatpe,isnull=isnul)
            colList.append(colitem)
        elif(len(colnames)<=4):
            colName=colnames[0][1:colnames[0].find(']')]
            #print(colName)
            datatpe=colnames[1][1:colnames[1].find(']')]
            #print(datatpe)
            isnul=colnames[2]
            if(isnul.find(",")>-1):
                isnul=isnul[0:-1]
            #print(isnul)
            colitem=ColumnNames(columnName=colName,dataType=datatpe,isnull=isnul)
            colList.append(colitem)
            #print(colitem.ColumnName)
    #print(colList)
                
    #print(colitem.DataType)
    '''
    for c in colList:
        print(c.ColumnName)
        print(c.DataType)
        print(c.IsNull)
    '''
    dd=DDL(TableName=tableName,columns=colList)
    DDLList.append(dd) 
#print(len(DDLList))
schemalist=[]
for tbl in DDLList:
    print("----"+tbl.TableName+"---")
    print("------------")
    
    tableSchema=tbl.TableName+"Schema"
    fieldadd=''
    for c in tbl.columns:
        colfield=''
        print(c.ColumnName) 
        print(c.DataType)
        print(c.IsNull)
        if(c.IsNull=="NULL"):
          nullable=True
        elif(c.IsNull=="NOT"):
           nullable=False
        if(c.DataType=='int'or c.DataType=='tinyint' or c.DataType=='bit'):
         colfield=f'StructField("{c.ColumnName}",IntegerType(),nullable={nullable}),\n' 
        elif( c.DataType=='money'):
         colfield=f'StructField("{c.ColumnName}",FloatType(),nullable={nullable}),\n'
        elif( c.DataType=='float'):
         colfield=f'StructField("{c.ColumnName}",FloatType(),nullable={nullable}),\n' 
        elif( c.DataType=='datetime'):
         colfield=f'StructField("{c.ColumnName}",DateType(),nullable={nullable}),\n'  
        elif( c.DataType=='date'):
          colfield=f'StructField("{c.ColumnName}",DateType(),nullable={nullable}),\n'
        else:
          colfield=f'StructField("{c.ColumnName}",StringType(),nullable={nullable}),\n'
          
        fieldadd=fieldadd+colfield
        
    opening="=StructType([\n"
    closing= "])"
    
    df=f'df = spark.read.option("sep", "|").schema({tableSchema}).format("csv").load("Files/AdventureWorksDW/{tbl.TableName}.csv")'
    dfwrite=f'df.write.mode("overwrite").format("delta").save("Tables/{tbl.TableName}")'
    overallSchema=tableSchema+opening+fieldadd+closing+"\n"+"\n"+df+"\n"+dfwrite+"\n"+"\n"
    schemalist.append(overallSchema)
    
with open("dmmodeling.txt","w",) as file:
   for dmls in schemalist:
    file.write(dmls)
    file.write("\n")
    
print(schemalist)

----DimAccount---
------------
AccountKey
int
NOT
ParentAccountKey
int
NULL
AccountCodeAlternateKey
int
NULL
ParentAccountCodeAlternateKey
int
NULL
AccountDescription
nvarchar
NULL
AccountType
nvarchar
NULL
Operator
nvarchar
NULL
CustomMembers
nvarchar
NULL
ValueType
nvarchar
NULL
CustomMemberOptions
nvarchar
NULL
----DimCurrency---
------------
CurrencyKey
int
NOT
CurrencyAlternateKey
nchar
NOT
CurrencyName
nvarchar
NOT
----DimCustomer---
------------
CustomerKey
int
NOT
GeographyKey
int
NULL
CustomerAlternateKey
nvarchar
NOT
Title
nvarchar
NULL
FirstName
nvarchar
NULL
MiddleName
nvarchar
NULL
LastName
nvarchar
NULL
NameStyle
bit
NULL
BirthDate
date
NULL
MaritalStatus
nchar
NULL
Suffix
nvarchar
NULL
Gender
nvarchar
NULL
EmailAddress
nvarchar
NULL
YearlyIncome
money
NULL
TotalChildren
tinyint
NULL
NumberChildrenAtHome
tinyint
NULL
EnglishEducation
nvarchar
NULL
SpanishEducation
nvarchar
NULL
FrenchEducation
nvarchar
NULL
EnglishOccupation
nvarchar
NULL
SpanishOccupation
nvarchar
NULL
F